In [207]:
try:
    import xmltodict
    print("module 'xmltodict' is installed")
except ModuleNotFoundError:
  get_ipython().system('pip install xmltodict')

from sklearn.linear_model import LassoCV, RidgeCV, HuberRegressor, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
# import statsmodels.api as sm
from functools import reduce

import requests
import xmltodict
import datetime

module 'xmltodict' is installed


In [2]:
# Web Call to Access and Download Data of a Single Station from AWDB Web Service (SOAP API)

def SOAP_Call(stationtriplets, elementCD, begindate, enddate):
  global xml, dict_of_xml, df
  # Create a dictionaries to store the data
  headers = {'Content-type': 'text/soap'}
  # current_dictionary = {}
  
  # Define Web Service URL
  URL = "https://wcc.sc.egov.usda.gov/awdbWebService/services?WSDL"

  # Define Parameters for SOAP Elements (getData:current and getCentralTendencyData:normals)
  SOAP_current = '''
  <?xml version="1.0" encoding="UTF-8"?>
  <SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:q0="http://www.wcc.nrcs.usda.gov/ns/awdbWebService" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <SOAP-ENV:Body>
      <q0:getData>
        <stationTriplets>STATIONTRIPLETS</stationTriplets>
        <elementCd>ELEMENTCD</elementCd>   
        <ordinal>1</ordinal>
        <duration>DAILY</duration>
        <getFlags>false</getFlags>
        <beginDate>BEGINDATE</beginDate>
        <endDate>ENDDATE</endDate>
        <alwaysReturnDailyFeb29>false</alwaysReturnDailyFeb29>   
      </q0:getData>
    </SOAP-ENV:Body>
  </SOAP-ENV:Envelope>

  '''.strip()
  #Read GetData documents - If <alwaysReturnDailyFeb29> is set to true, will set a null for every non leap year on the 29th,  
  #which breaks this request when selecting date ranges that include Feb 29.
  #Possible element codes: PREC, WTEQ (Water Equivalent/SWE)
  
  # Post SOAP Elements to AWDB Web Service and process results - getData
  SOAP_current = SOAP_current.replace("ELEMENTCD", elementCD)
  SOAP_current = SOAP_current.replace("STATIONTRIPLETS", stationtriplets)
  SOAP_current = SOAP_current.replace("BEGINDATE", begindate)
  SOAP_current = SOAP_current.replace("ENDDATE", enddate)

  #Send request to server and receive xml document
  xml = requests.post(URL, data=SOAP_current, headers=headers)

  #convert xml document to a dictionary, extract values putting them in a dataframe.  XML's aren't the easiest to parse and extract data from, so this is a nice work around.
  dict_of_xml = xmltodict.parse(xml.text)
  df = dict_of_xml['soap:Envelope']['soap:Body']['ns2:getDataResponse']['return']['values']

  #Null values are given as OrderedDictionaries with lots of text, while actual values are given as strings.  This converts all the OrderedDictionaries into actual null/none values, and converts all values that were given as strings into float numbers.
  df = pd.DataFrame(map(lambda i: float(i) if type(i) == str else None, df))

  #Since invidual dates aren't associated with the values in the xml document, have to create a range of dates bw the begindate and endate, which is then added to the dataframe.
  df['Date'] = pd.date_range(begindate,enddate,freq='d')
  df.columns = [f'{elementCD}','Date']
  df.set_index('Date', inplace=True)

  return df


# Function to download data from multiple stations at a time from AWDB Web Service
# Web service request will except multiple stations in a single request, so this is definitely not the best way to do this as it sends multiple requests combining each into a single pandas dataframe.  

def getData(stations, parameter_of_interest, begindate, enddate): 
  try:
    data_singleDF = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), [SOAP_Call(stationtriplets=j,elementCD=parameter_of_interest,begindate=begindate,enddate=enddate) for j in stations])  
    data_singleDF.columns = [f'{j}' for j in stations]
    return data_singleDF
    
  except ValueError:
    print('ValueError occured. Data does not exist for the specific date of either (or both) the begindate or enddate.  Select a different date(s), or enter temporary value(s) into DMP')

  # except UnboundLocalError:
  #   print('Data does not exist for the specific date of either (or both) the begindate or enddate.  Select a different date(s), or enter temporary value(s) into DMP')

  except KeyError:
    print('KeyError occurred.')  
    print('Possible reasons for this error include:')
    print('\n')
    print('Stations are not in brackets. For example, \'401:OR:SNTL\' should instead be entered as [\'401:OR:SNTL\'].')
    print('Stations are incorrectly called or mispelled.  Check that the correct station ID, state, and network are correct.')
    print('Parameter was mispelled. For example, SWE was entered instead of WTEQ')
    print('Date is not in the correct DD/MM/YYYY format.')


In [208]:
#Function that assigns the regression model to be used and its parameters

def regressionModel(regression_model):
  if regression_model == 'Lasso':
    regr = LassoCV(alphas=(0.001,0.01,0.1,1,10,100,1000))
    return regr
  elif regression_model == 'Ridge':
    regr = RidgeCV(alphas=(0.001,0.01,0.1,1,10,100,1000))
  elif regression_model == 'Linear':
    regr = LinearRegression()
    return regr
  elif regression_model == 'SVM': 
    regr = svm.SVR(kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
    return regr
  elif regression_model == 'Huber':
    regr = HuberRegressor()
    return regr
  elif regression_model == 'GradientBoost':
    regr = GradientBoostingRegressor(random_state=0, n_estimators=100)
    return regr
  elif regression_model == 'AdaBoost':
    regr = AdaBoostRegressor(random_state=0, n_estimators=100)
    return regr
  elif regression_model == 'Random Forest':
    regr = RandomForestRegressor(n_estimators=100)
    return regr
  else:
    print('Choose either Lasso, Ridge, SVM, Huber, GradientBoost, AdaBoost, or Random Forest')

In [16]:
def test(**kwargs):
    for key, value in kwargs.items():
        print ("%s SWE is %s" %(key, value))

In [17]:
test(adin=3.2)

adin SWE is 3.2


In [86]:
stationparameter_list = [{'abc':123}, {'edf':456}]

In [42]:
print(type(list(stationparameter_list[0].values())[0]))
print(type([stationparameter_list[0].values()]))

<class 'int'>
<class 'list'>


In [44]:
list(stationparameter_list[0].keys())[0]

'abc'

In [91]:
for i in stationparameter_list:
    print(i)
    print(list(i.values())[0])

{'abc': 123}
123
{'edf': 456}
456


In [85]:
bla

[            WTEQ
 Date            
 2020-01-01   4.3
 2020-01-02   5.1
 2020-01-03   5.3
 2020-01-04   5.4
 2020-01-05   5.5
 ...          ...
 2021-02-25   NaN
 2021-02-26   NaN
 2021-02-27   NaN
 2021-02-28   NaN
 2021-03-01  14.5
 
 [426 rows x 1 columns],
             PREC
 Date            
 2020-01-01   8.8
 2020-01-02   9.5
 2020-01-03   9.8
 2020-01-04   9.9
 2020-01-05   9.9
 ...          ...
 2021-02-25  20.9
 2021-02-26  21.5
 2021-02-27  22.0
 2021-02-28  22.0
 2021-03-01  22.5
 
 [426 rows x 1 columns]]

In [101]:
stationparameter = [{'302:OR:SNTL':'WTEQ'}, {'653:OR:SNTL':'PREC'}]

In [102]:
begindate = '01/01/2020'
enddate = '02/01/2020'

data = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), [SOAP_Call(stationtriplets=list(i.keys())[0],elementCD=list(i.values())[0],begindate=begindate,enddate=enddate) for i in stationparameter])  
data.columns = [f'{j}' for j in stationparameter]


In [103]:
data

,{'302:OR:SNTL': 'WTEQ'},{'653:OR:SNTL': 'PREC'}
Date,,
2020-01-01,4.3,8.8
2020-01-02,5.1,9.5
2020-01-03,5.3,9.8
2020-01-04,5.4,9.9
2020-01-05,5.5,9.9
2020-01-06,5.6,10.2
2020-01-07,6.0,10.5
2020-01-08,6.3,10.5
2020-01-09,6.5,10.8


In [140]:
 def getData(stationparameter_list, begindate, enddate):   

        
#If you want each station:parameter request returned as an item in a list:       
#     data = []  
#     for i in range(len(stationparameter_list)):     
#         data.append(SOAP_Call(list(stationparameter_list[i].keys())[0], list(stationparameter_list[i].values())[0], begindate, enddate))
#     return data
        
    
#     def getData(stations, parameter_of_interest, begindate, enddate): 
      try:
        data = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), [SOAP_Call(stationtriplets=list(i.keys())[0],elementCD=list(i.values())[0],begindate=begindate,enddate=enddate) for i in stationparameter_list])  
        data.columns = [f'{j}' for j in stationparameter_list]
        return data

      except ValueError:
        print('ValueError occured. Data does not exist for the specific date of either (or both) the begindate or enddate.  Select a different date(s), or enter temporary value(s) into DMP')

      # except UnboundLocalError:
      #   print('Data does not exist for the specific date of either (or both) the begindate or enddate.  Select a different date(s), or enter temporary value(s) into DMP')

      except KeyError:
        print('KeyError occurred.')  

In [163]:
bla = getData([{'302:OR:SNTL':'WTEQ'}, {'302:OR:SNTL':'WTEQ'}], '01/01/2020','03/01/2021')
bla

,{'302:OR:SNTL': 'WTEQ'},{'302:OR:SNTL': 'WTEQ'}
Date,,
2020-01-01,4.3,4.3
2020-01-02,5.1,5.1
2020-01-03,5.3,5.3
2020-01-04,5.4,5.4
2020-01-05,5.5,5.5
...,...,...
2021-02-25,NaN,NaN
2021-02-26,NaN,NaN
2021-02-27,NaN,NaN


In [164]:
foo = [{'302:OR:SNTL':'WTEQ'}, {'302:OR:SNTL':'WTEQ'}]


In [172]:
coo = []
for i in foo:
    print (i)
    coo.append(list(i.values())[0])

{'302:OR:SNTL': 'WTEQ'}
{'302:OR:SNTL': 'WTEQ'}


In [175]:
[list(i.values())[0] for i in foo]

['WTEQ', 'WTEQ']

In [76]:
# %%timeit
getData(['302:OR:SNTL'], 'WTEQ','01/01/2020','03/01/2021')

ConnectionError: HTTPSConnectionPool(host='wcc.sc.egov.usda.gov', port=443): Max retries exceeded with url: /awdbWebService/services?WSDL (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000153FAD13F10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [77]:
# %%timeit
SOAP_Call('302:OR:SNTL','WTEQ','01/01/2020','03/01/2021')

ConnectionError: HTTPSConnectionPool(host='wcc.sc.egov.usda.gov', port=443): Max retries exceeded with url: /awdbWebService/services?WSDL (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000153FADD8490>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [209]:
class RegressionFun():

  '''
  stations: list of stationtriplets, with the response station in the first position followed by the predictor station(s).  Ex. ['817:WA:SNTL', '711:WA:SNTL', '975:WA:SNTL']
  parameter_of_interest: can choose 'WTEQ', 'SNWD', 'PREC', 'PRCP', 'TAVG', 'TOBS', and others (https://www.nrcs.usda.gov/wps/portal/wcc/home/dataAccessHelp/webService/webServiceReference/!ut/p/z1/jc9NDoIwEAXgs3CCPqCWuqxNBFIiqdCAsyFdkSaKLozn1xg2LmyY3STfmx9GbGS0-FeY_TPcF3_99BcSk5Sapy1HU576HFactTtWJkMKNnxBnUlVyQ6m1JxD7XXRN10BcwCjLXn8KbUxHwEUHz8w-l0BWWew2hre7pBDpyuIvRgFvVhB5IrHzbkRoZ5VkrwBnC-XyQ!!/?1dmy&current=true&urile=wcm%3apath%3a%2Fwcc%2Bcontent%2Fhome%2Fdata%2Baccess%2Bhelp%2Fweb%2Bservice%2Fweb%2Bservice%2Breference#elementCodes).
  '''

  def __init__(self, stationparameterpairs, begindate, enddate): #, stations, parameter_of_interest):

    self.stationparameterpairs = stationparameterpairs
    self.stations = [list(i.keys())[0] for i in stationparameterpairs]
    self.parameters = [list(i.values())[0] for i in stationparameterpairs]
    
    #Download Data from AWDB Web Service
    self.begindate = begindate
    self.enddate = enddate
    self.data = getData(self.stationparameterpairs, begindate, enddate).dropna()
    
    
  def train_model(self, regression_model, test_size):
    
    '''
    Function checks model fit on train and test sets.  Use to check which stations result in the best fitting model.  Once the best model is found, it can be used in the make_predictions function to predict null values.
    
    begindate: non-null date in 'mm/dd/yyyy' format
    enddate: non-null date in 'mm/dd/yyyy' format
    regression_model: can be 'Ridge', 'Lasso', 'Huber', 'SVM', 'Random Forest', 'AdaBoost', 'GradientBoost'
    test_size: value between 0 and 1 that defines the percentage of the data to be used as the test_size
    '''

    #Define Targets and Features (e.g. Response and Predictor Variables)
    target = self.data.iloc[:,0]
    features = self.data.iloc[:,1:]

    #Split into training and test sets
    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=test_size, shuffle=False)
  
    #Choose Regression Model:
    regr = regressionModel(regression_model)
  
    #Fit model on training set
    regr.fit(features_train, target_train)

    self.regr = regr

    #Run predictions on training features and test features
    target_train_pred = regr.predict(features_train)
    target_test_pred = regr.predict(features_test)

    #Print Root Mean Square Error for training and test sets:
    print('RMSE for training set', mean_squared_error(target_train, target_train_pred))
    print('RMSE for test set', mean_squared_error(target_test, target_test_pred))

    #Predictions plot

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y = target_train,
        x = target_train.index,
        mode = 'lines',
        name = 'Training Data'
    ))

    fig.add_trace(go.Scatter(
        y = target_train_pred,
        x = target_train.index,
        mode = 'lines',
        name = 'Model Predictions on Training Data'
    ))

    fig.add_trace(go.Scatter(
        y = target_test,
        x = target_test.index,
        mode = 'lines',
        name = 'Test Data'
    ))

    fig.add_trace(go.Scatter(
        y = target_test_pred,
        x = target_test.index,
        mode = 'lines',
        name = 'Model Predictions on Test Data'
    ))

    fig.update_yaxes(title_text = f"{self.parameters[0]}")
    fig.update_xaxes(title_text = 'Date') 

    fig.update_layout(
        showlegend=True,
        height=450,
        width=1200 
    )
    fig.show()

    # # # # # Regression Plot # # # # #

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
      x=features_train.iloc[:,0],
      y=target_train,
      mode='markers',
      hovertext = features_train.index,
      name =  'Response vs. Predictors'
    ))

    fig2.add_trace(go.Scatter(
      x=features_train.iloc[:,0],
      y=pd.DataFrame(target_train_pred.tolist()).iloc[:,0],
      mode='lines',
      hovertext = features_train.index,
      name = 'Model Fit'  
    ))
    
    fig2.update_xaxes(title_text = f'{self.stations[0]} {self.parameters[0]}')
    fig2.update_yaxes(title_text = f'{self.stationparameterpairs[1:]}')
    fig2.update_layout(
        showlegend=True,
        height=450,
        width=1200 
    )
    fig2.show()

    return regr
      
      
  def make_predictions(self, predict_begindate, predict_enddate):

    #Download Data from AWDB Web Service
    predict_data  = getData(self.stationparameterpairs, predict_begindate, predict_enddate)

    predict_target = predict_data.iloc[:,0].fillna(0)
    predict_features = predict_data.iloc[:,1:].fillna(0)

    #Run predictions
    predictions = self.regr.predict(predict_features)
    
    #Plot predictions 
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        y = predict_target,
        x = predict_target.index,
        mode = 'lines',
        name = f'{self.stations[0]} {self.parameters[0]}'
    )) 

    fig.add_trace(go.Scatter(
        y = predictions,
        x = predict_features.index,
        mode = 'lines',
        name = 'Model Predictions'
    )) 

    fig.update_layout(
      showlegend=True,
      height=650,
      width=1400,
      title={
          'text': "Model Predictions",
          'xanchor': 'center',
          'yanchor': 'top',
          'y':0.9,
          'x':0.4},
      xaxis_title = "Date",
      yaxis_title = f"{self.stationparameterpairs[0]}"
    )

    fig.show()  

In [210]:
apple = RegressionFun([{'302:OR:SNTL':'WTEQ'},  {'302:OR:SNTL':'SNWD'}, {'302:OR:SNTL':'TAVG'}, {'653:OR:SNTL':'WTEQ'}], '01/01/2010','02/01/2020')

In [153]:
# apple = RegressionFun([{'302:OR:SNTL':'WTEQ'}, {'302:OR:SNTL':'SNWD'}])

In [180]:
apple.parameters

['WTEQ', 'WTEQ']

In [211]:
apple.train_model('Linear', 0.3)

RMSE for training set 3.570029415020483
RMSE for test set 8.112468786361747


LinearRegression()

In [212]:
apple.make_predictions('02/01/2020', '03/01/2020')